In [6]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [7]:
import sagemaker
from sagemaker import get_execution_role
import transformers
import torch

import boto3
import json
import time

role = get_execution_role()
sess = sagemaker.Session()

account = sess.account_id()
bucket = sess.default_bucket() # or use your own custom bucket name
prefix = 'mme-cv-benchmark'

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")

model_name = "roberta-large" #"roberta-large" #"gpt2" #'bert-base-uncased' #'gpt2' #change the model name to benchmark different CV models
instance_type = 'ml.g4dn.2xlarge'

OSError: /home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/lib/../../nvidia/cublas/lib/libcublas.so.11: symbol cublasLtHSHMatmulAlgoInit, version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference

In [ ]:
account_id_map = {
    'us-east-1': '785573368785',
    'us-east-2': '007439368137',
    'us-west-1': '710691900526',
    'us-west-2': '301217895009',
    'eu-west-1': '802834080501',
    'eu-west-2': '205493899709',
    'eu-west-3': '254080097072',
    'eu-north-1': '601324751636',
    'eu-south-1': '966458181534',
    'eu-central-1': '746233611703',
    'ap-east-1': '110948597952',
    'ap-south-1': '763008648453',
    'ap-northeast-1': '941853720454',
    'ap-northeast-2': '151534178276',
    'ap-southeast-1': '324986816169',
    'ap-southeast-2': '355873309152',
    'cn-northwest-1': '474822919863',
    'cn-north-1': '472730292857',
    'sa-east-1': '756306329178',
    'ca-central-1': '464438896020',
    'me-south-1': '836785723513',
    'af-south-1': '774647643957'
}

In [2]:
region = boto3.Session().region_name
if region not in account_id_map.keys():
    raise("UNSUPPORTED REGION")

NameError: name 'boto3' is not defined

In [30]:
if model_name == "bert-base-uncased":
    model = transformers.BertModel.from_pretrained(model_name, torchscript=True)
elif model_name == "roberta-large":
    model = transformers.RobertaModel.from_pretrained(model_name, torchscript=True)
elif model_name == "distilbert-base-uncased":
    model = transformers.DistilBertModel.from_pretrained(model_name, torchscript=True)
else:
    raise ("THIS MODEL HAS NOT BEEN TESTED")
    
example_input = (
        torch.randint(1000, (32, 128)),
        torch.zeros(32, 128, dtype=torch.int)
    )

model.eval()


traced_script_module = torch.jit.trace(model, example_input)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Packaging Pytorch model for Triton sever on SageMaker

**Note**: SageMaker expects the model tarball file to have a top level directory with the same name as the model defined in the `config.pbtxt`.

```
model_name
├── 1
│   └── model.pt
└── config.pbtxt
```

In [31]:
pytorch_model_path = f"{model_name}-pt/{model_name}/1"

!mkdir -p $pytorch_model_path

traced_script_module.save(f"{pytorch_model_path}/model.pt")

In [32]:
%%writefile {model_name}-pt/{model_name}/config.pbtxt
platform: "pytorch_libtorch"
max_batch_size: 32
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [128]
  },
  {
    name: "INPUT__1"
    data_type: TYPE_INT32
    dims: [128]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [128, 768]
  },
  {
    name: "1634__1"
    data_type: TYPE_FP32
    dims: [768]
  }
]
instance_group {
  count: 1
  kind: KIND_GPU
}
dynamic_batching {
  preferred_batch_size: 16
}

Overwriting roberta-large-pt/roberta-large/config.pbtxt


## Generate a TensorRT model

1. Export the pre-trained into an ONNX file. ONNX is intemedia file format which is framework agnostic. It works with models in TF, PyTorch and more.

2. Spin up a Sagemaker processing job to convert ONNX model to a TensorRT model plan. You will export the weights of your model from the framework and load them into your tensorRT network.

### <span style="color:red">For TensorRT models, The hosting instance type must match the instance type the model is compiled on. </span>

In [ ]:
onnx_model_path = "onnx-models"
!mkdir -p {onnx_model_path}

torch.onnx.export(
    model,
    example_input,
    f"{onnx_model_path}/{model_name}.onnx",
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=["INPUT__0", "INPUT__1"],
    output_names=["OUTPUT__0"],
    dynamic_axes={"INPUT__0": [0, 1], "INPUT__1": [0, 1], "OUTPUT__0": [0]},
)

In [ ]:
%%writefile trt_compile.sh
#!/bin/bash
mkdir /opt/ml/processing/output/
/usr/src/tensorrt/bin/trtexec $@

In [ ]:
base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
mme_triton_image_uri = f"{account_id_map[region]}.dkr.ecr.{region}.{base}" + \
            "/sagemaker-tritonserver:22.07-py3"
print(mme_triton_image_uri)

In [21]:
import boto3
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput, Processor
from sagemaker import get_execution_role

import uuid

region = boto3.session.Session().region_name

s3_output = f's3://{bucket}/{prefix}/{model_name}/outputs/'


compilation_job = ScriptProcessor(
    image_uri=mme_triton_image_uri,
    command=["/bin/bash"],
    instance_type=instance_type,
    volume_size_in_gb=100,
    instance_count=1,
    base_job_name=f"{prefix}-{model_name.replace('_', '')}",
    role=role,  
)

compilation_job.run(
    code = "trt_compile.sh",
    inputs=[ProcessingInput(
            source=f"{onnx_model_path}/{model_name}.onnx",
            destination='/opt/ml/processing/input_data/'),
           ],
    outputs=[ProcessingOutput(source="/opt/ml/processing/output/", 
                         destination=s3_output)],
    arguments=[f"--onnx=/opt/ml/processing/input_data/{model_name}.onnx", 
               "--saveEngine=/opt/ml/processing/output/model.plan", 
               "--explicitBatch", 
               "--minShapes=INPUT__0:1x128,INPUT__1:1x128",
               "--optShapes=INPUT__0:16x128,INPUT__1:16x128",
               "--maxShapes=INPUT__0:16x128,INPUT__1:16x128",
               "--fp16",
               "--verbose"
              ],
        wait=False
)


Job Name:  mme-cv-benchmark-distilbert-base-uncase-2022-11-23-14-11-44-485
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-376678947624/mme-cv-benchmark-distilbert-base-uncase-2022-11-23-14-11-44-485/input/input-1/distilbert-base-uncased.onnx', 'LocalPath': '/opt/ml/processing/input_data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-376678947624/mme-cv-benchmark-distilbert-base-uncase-2022-11-23-14-11-44-485/input/code/trt_compile.sh', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-376678947624/mme-cv-benchmark/distilbert-base-uncased/

In [22]:
job_waiter = sm_client.get_waiter('processing_job_completed_or_stopped')
print("Waiting for job to complete")
job_waiter.wait(ProcessingJobName=compilation_job.latest_job.job_name)
print("Job completed")

Waiting for job to complete


WaiterError: Waiter ProcessingJobCompletedOrStopped failed: Waiter encountered a terminal failure state: For expression "ProcessingJobStatus" we matched expected path: "Failed"

## Packaging TensorRT model for Triton sever on SageMaker

**Note**: SageMaker expects the model tarball file to have a top level directory with the same name as the model defined in the `config.pbtxt`.


```
model_name
├── 1
│   └── model.plan
└── config.pbtxt
```

In [10]:
!mkdir -p {model_name}-trt/{model_name}/1/
!aws s3 cp {s3_output}model.plan {model_name}-trt/{model_name}/1/

download: s3://sagemaker-us-west-2-376678947624/mme-cv-benchmark/roberta-large/outputs/model.plan to roberta-large-trt/roberta-large/1/model.plan


In [11]:
%%writefile {model_name}-pt/{model_name}/config.pbtxt
platform: "tensorrt_plan"
max_batch_size: 32
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [128]
  },
  {
    name: "INPUT__1"
    data_type: TYPE_INT32
    dims: [128]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [128, 1024] 
  },
  {
    name: "3192"
    data_type: TYPE_FP32
    dims: [1024]
  }    
]
instance_group {
  count: 1
  kind: KIND_GPU
}
dynamic_batching {
  preferred_batch_size: 16
}

Overwriting roberta-large-pt/roberta-large/config.pbtxt


## Load local triton server

In [9]:
import os

cur_dir = os.getcwd()
print(f"docker run --gpus=all --rm -p8000:8000 -p8001:8001 -p8002:8002 -v{cur_dir}/{model_name}-pt:/models nvcr.io/nvidia/tritonserver:22.07-py3 tritonserver --model-repository=/models")

docker run --gpus=all --rm -p8000:8000 -p8001:8001 -p8002:8002 -v/home/ec2-user/SageMaker/gpu-test/distilbert-base-uncased-pt:/models nvcr.io/nvidia/tritonserver:22.07-py3 tritonserver --model-repository=/models


In [15]:
import numpy as np
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype

def tokenize_text(text, model_name):
    if model_name == 'roberta-large':
        enc = transformers.RobertaTokenizer.from_pretrained(model_name)
    elif model_name == 'bert-base-uncased':
        enc = transformers.BertTokenizer.from_pretrained(model_name)
    elif model_name == 'distilbert-base-uncased':
        enc = transformers.DistilBertTokenizer.from_pretrained(model_name)
    else:
        pass

    encoded_text = enc(text, padding="max_length", max_length=128)
    return np.array([encoded_text["input_ids"]], dtype=np.int32), np.array([encoded_text["attention_mask"]], dtype=np.int32)

In [17]:
text_triton = "Triton Inference."
input_ids, attention_mask = tokenize_text(text_triton, model_name)

client = httpclient.InferenceServerClient(url="localhost:8000")

inputs = []

inputs.append( httpclient.InferInput("INPUT__0", [1,128], datatype="INT32"))
inputs[0].set_data_from_numpy(input_ids)
inputs.append( httpclient.InferInput("INPUT__1", [1,128], datatype="INT32"))
inputs[1].set_data_from_numpy(attention_mask)
# inputs.set_data_from_numpy(transformed_img, binary_data=True)

outputs = []
outputs.append(httpclient.InferRequestedOutput("OUTPUT__0"))
# outputs.append(httpclient.InferRequestedOutput("1634__1"))

# Querying the server

for x in range(1000):
    results = client.infer(model_name=model_name, inputs=inputs, outputs=outputs)
    print(results.as_numpy('OUTPUT__0')[0][:5])

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

...

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.

[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.06199513 ... -0.1395507   0.14923914
   0.14800404]
 [ 0.3904745   0.1533219  -0.09512854 ...  0.09139397  0.18775365
   0.2937125 ]
 [-0.82730585 -0.74026984 -0.5026222  ...  0.39359596  0.20664102
  -0.24987502]]
[[-0.42516118 -0.21769695 -0.2412315  ... -0.09374391  0.2964332
   0.35510576]
 [ 0.9941633  -0.1709471   0.15332487 ... -0.04615122  0.33930314
   0.05280082]
 [-0.14350739 -0.38533992 -0.